# SISTEMA DE RECOMENDAÇÃO DE FILMES

## A. Introdução
• Detalhar objetivo escolhido para trabalhar neste projeto juntamente com descrição da
base de dados. Pesquise trabalhos na literatura que discutam o tema escolhido. Para
trabalhos acadêmicos, acesse https://scholar.google.com.br/. Guarde as referências
estudadas para citá-las no projeto.


O objetivo do nosso projeto é fazer a predição da receita de um filme/um recomendador com base em determinadas variáveis. A partir disso, guiaremos nosso projeto, utilizando técnicas de Machine Learning para alcançar esse objetivo. 




## B. Minerando Dados e Características do Dataset

• Se necessário, faça filtro na base de dados tanto de linhas como de colunas em prol do
objetivo traçado anteriormente.
• Descreva as variáveis finais que serão utilizadas a partir deste ponto.
• Faça análise descritiva detalhada das variáveis, norteado pelo objetivo do problema.


### Importando as bibliotecas

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

### Lendo os arquivos

In [3]:
import os
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/beatrizcf/2 Semestre - DP/CDADOS/Classificador_Filmes


Primeiramente vamos ler os arquivos que contem os metadados dos filmes e as avaliacoes dos usuarios

In [21]:
metadados = pd.read_csv('movies_metadata.csv')
metadados.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [44]:
metadados.loc[metadados['id']=='1029']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [42]:
metadados.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [50]:
metadados['id'] = metadados['id'].astype('int')

ValueError: invalid literal for int() with base 10: '1997-08-20'

In [47]:
ratings = pd.read_csv('ratings_small.csv')
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [48]:
links = pd.read_csv('links_small.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


Para facilitar a visualizacao, vamos 

In [46]:
#ratings = ratings.rename(columns={'movieId': 'id'})
ratings.dtypes
#ratings = pd.merge(ratings, metadados, on='id')

userId         int64
id             int64
rating       float64
timestamp      int64
dtype: object

### Informações sobre o dataset

O dataset a ser analisado foi obtido através do TMDB API e conta com um enorme conjunto de dados. Os filmes disponíveis nesse dataset são correespondentes aos filmes listados no *MovieLens Latest Full Dataset*, composta por   26 milhoes de avaliacoes em 45,000 filmes de 27,000 users. A seguir vamos analisar as variáveis disponíveis nesse dataset.

In [5]:
dados_md.columns


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

#### Variáveis disponíveis

* **adult:** Indicates if the movie is X-Rated or Adult.
* **belongs_to_collection:** A stringified dictionary that gives information on the movie series the particular film belongs to.
* **budget:** The budget of the movie in dollars.
* **genres:** A stringified list of dictionaries that list out all the genres associated with the movie.
* **homepage:** The Official Homepage of the move.
* **id:** The ID of the move.
* **imdb_id:** The IMDB ID of the movie.
* **original_language:** The language in which the movie was originally shot in.
* **original_title:** The original title of the movie.
* **overview:** A brief blurb of the movie.
* **popularity:** The Popularity Score assigned by TMDB.
* **poster_path:** The URL of the poster image.
* **production_companies:** A stringified list of production companies involved with the making of the movie.
* **production_countries:** A stringified list of countries where the movie was shot/produced in.
* **release_date:** Theatrical Release Date of the movie.
* **revenue:** The total revenue of the movie in dollars.
* **runtime:** The runtime of the movie in minutes.
* **spoken_languages:** A stringified list of spoken languages in the film.
* **status:** The status of the movie (Released, To Be Released, Announced, etc.)
* **tagline:** The tagline of the movie.
* **title:** The Official Title of the movie.
* **video:** Indicates if there is a video present of the movie with TMDB.
* **vote_average:** The average rating of the movie.
* **vote_count:** The number of votes by users, as counted by TMDB.


Analisando brevemente as variaveis disponiveis, nota-se que algumas delas nao sao ou nao trazem informacoes  relevantes ao nosso projeto. Tais como, *id, imdb_id, original_title, poster_path, video*

In [12]:
dados.adult.value_counts()

False                                                                                                                             45454
True                                                                                                                                  9
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 - Written by Ørnås                                                                                                                   1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: adult, dtype: int64

## C. Modelos de Predição
• Descreva e justifique sua escolha de modelos de predição. Exemplos de modelos (os
exemplos a seguir são utilizados neste trabalho, mas você pode usar outros que fizerem
mais sentido para o seu problema):
o Modelo de predição pela média (sem uso de variável explicativa);
o Modelo de regressão linear (Multiple Linear Regression);
o Modelo de árvores de regressão (Decision Tree Regression).

## D. Processo e Estatísticas de Validação
• Para os modelos preditivos que foram desenvolvidos no item anterior, é necessário
calcular medidas que informam a performance de cada modelo ajustado. Assim, para
cada modelo preditivo, faça:
o Divida a base de dados na parte treinamento e na parte teste. Use a parte
treinamento para estimar cada modelo preditivo.
o Estude as medidas coeficiente de determinação (R2) e raiz do erro médio
quadrático (RMS) descritas aqui. Calcule essas medidas tanto para a parte dos
dados de treinamento como para a parte dos dados teste.
o Discuta se essas duas medidas se comportam de forma semelhante para as duas
partes de dados. Leia o texto disponível aqui para compreender overfitting e
underfitting e, com isso, refinar senso crítico para discutir sobre as medidas
calculadas.
o Extra: Faça o processo de Validação Cruzada utilizando também 10 ciclos e
calcule a performance média e desvio padrão das duas medidas R2 e RMS tanto
para a parte treinamento como para a parte teste. Discuta com riqueza de
detalhes.

## E. Conclusão
• Faça conclusão final com detalhes levando em consideração todas as interpretações
realizadas no decorrer do projeto.

## F. Referências Bibliográficas
• Todas as pesquisas feitas e estudadas que foram relevantes para o desenvolvimento
devem ser citadas no projeto.

* https://towardsdatascience.com/how-to-build-a-simple-recommender-system-in-python-375093c3fb7d


In [17]:
ratings = pd.read_csv('ratings_small.csv')

In [18]:
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131
